# Data Cleaning and Preparation

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Handling Missing Data

In [ ]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

In [ ]:
string_data[0] = None
string_data.isnull()

### Filtering Out Missing Data

In [ ]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

In [ ]:
data[data.notnull()]

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
data
cleaned

In [ ]:
data.dropna(how='all')

In [ ]:
data[4] = NA
data
data.dropna(axis=1, how='all')

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df
df.dropna()
df.dropna(thresh=2)

### Filling In Missing Data

In [ ]:
df.fillna(0)

In [ ]:
df.fillna({1: 0.5, 2: 0})

In [ ]:
_ = df.fillna(0, inplace=True)
df

In [ ]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df
df.fillna(method='ffill')
df.fillna(method='ffill', limit=2)

In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

## Data Transformation

### Removing Duplicates

In [ ]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

In [ ]:
data.duplicated()

In [ ]:
data.drop_duplicates()

In [ ]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

In [ ]:
data.drop_duplicates(['k1', 'k2'], keep='last')

### Transforming Data Using a Function or Mapping

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

In [ ]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [ ]:
lowercased = data['food'].str.lower()
lowercased
data['animal'] = lowercased.map(meat_to_animal)
data

In [ ]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

### Replacing Values

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

In [ ]:
data.replace(-999, np.nan)

In [ ]:
data.replace([-999, -1000], np.nan)

In [ ]:
data.replace([-999, -1000], [np.nan, 0])

In [ ]:
data.replace({-999: np.nan, -1000: 0})

### Renaming Axis Indexes

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [ ]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

In [ ]:
data.index = data.index.map(transform)
data

In [ ]:
data.rename(index=str.title, columns=str.upper)

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

In [ ]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

### Discretization and Binning

In [2]:
#12 ages
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [22]:
# cut into categories - number is open end on left by default
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats[:3]

[(18, 25], (18, 25], (18, 25]]
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [14]:
# Categorical object contain ordered array of categories.  Original ages are lost
print(cats)

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]


In [19]:
# change 1st value to 21 yields identical categorical
ages_altered = [21, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
cats_altered = pd.cut(ages_altered, bins)
cats.categories == cats_altered.categories

array([ True,  True,  True,  True])

In [8]:
cats.codes, cats.categories, pd.value_counts(cats)

(array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8),
 IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
               closed='right',
               dtype='interval[int64]'),
 (18, 25]     5
 (35, 60]     3
 (25, 35]     3
 (60, 100]    1
 dtype: int64)

In [20]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

In [25]:
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.72, 0.93], (0.3, 0.51], (0.72, 0.93], (0.095, 0.3], (0.095, 0.3], ..., (0.095, 0.3], (0.3, 0.51], (0.3, 0.51], (0.72, 0.93], (0.72, 0.93]]
Length: 20
Categories (4, interval[float64]): [(0.095, 0.3] < (0.3, 0.51] < (0.51, 0.72] < (0.72, 0.93]]

In [28]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4, precision=2)  # Cut into quartiles
cats
pd.value_counts(cats)

(0.68, 3.26]       250
(-0.022, 0.68]     250
(-0.63, -0.022]    250
(-3.76, -0.63]     250
dtype: int64

In [29]:
cats.categories

IntervalIndex([(-3.76, -0.63], (-0.63, -0.022], (-0.022, 0.68], (0.68, 3.26]],
              closed='right',
              dtype='interval[float64]')

In [36]:
# set unequal counts in the quartiles
a = pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.], precision=2)
a.value_counts()

(-3.76, -1.3]     100
(-1.3, -0.022]    400
(-0.022, 1.34]    400
(1.34, 3.26]      100
dtype: int64

### Detecting and Filtering Outliers

In [38]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.042878,-0.035881,-0.034085,0.014845
std,0.988756,0.961679,0.988039,1.015878
min,-3.333767,-2.901831,-3.428254,-3.645860
25%,-0.605292,-0.702176,-0.731670,-0.664374
50%,0.028040,-0.062375,-0.068581,0.038757
75%,0.695298,0.664260,0.625698,0.721104
max,3.525865,2.611678,3.366626,2.763474


In [49]:
data.columns[2]

2

In [50]:
col = data[2]
col[np.abs(col) > 3]

12    -3.428254
389    3.366626
885   -3.108915
Name: 2, dtype: float64

In [66]:
# reducing rows. any row by columns

type(data[(np.abs(data) > 3)])
data[(np.abs(data) > 3)].any(axis=0)

0     True
1    False
2     True
3     True
dtype: bool

In [67]:
# reducing columns. any column by row

type(data[(np.abs(data) > 3)])
data[(np.abs(data) > 3)].any(axis=1)

0      False
1      False
2      False
3      False
4      False
       ...  
995    False
996    False
997    False
998    False
999    False
Length: 1000, dtype: bool

In [79]:
type((np.abs(data) > 3).any(1))

pandas.core.series.Series

In [97]:
# when using a Series as a boolean mask, the mask index must be a superset of the index of the dataframe

data[(np.abs(data) > 3).any(axis=1)]

,0,1,2,3
12,0.326045,0.425384,-3.428254,-0.296336
76,-3.184377,1.369891,-1.074833,-0.089937
297,0.208011,-0.150923,-0.362528,-3.548824
389,0.193299,1.397822,3.366626,-2.372214
536,3.525865,0.283070,0.544635,0.462204
556,-0.450721,-0.080332,0.599947,-3.645860
779,-3.333767,-1.240685,-0.650855,0.076254
860,0.344072,0.581893,-1.116332,-3.018842
885,-0.555434,-0.048478,-3.108915,1.117755


In [98]:
# cap outliers at +/-3
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.042871,-0.035881,-0.033915,0.016059
std,0.985366,0.961679,0.985127,1.011910
min,-3.000000,-2.901831,-3.000000,-3.000000
25%,-0.605292,-0.702176,-0.731670,-0.664374
50%,0.028040,-0.062375,-0.068581,0.038757
75%,0.695298,0.664260,0.625698,0.721104
max,3.000000,2.611678,3.000000,2.763474


In [ ]:
np.sign(data).head()

### Permutation and Random Sampling

In [100]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([0, 3, 2, 1, 4])

In [101]:
df
df.take(sampler)

,0,1,2,3
0,0,1,2,3
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7
4,16,17,18,19


In [103]:
df.iloc[sampler]

,0,1,2,3
0,0,1,2,3
3,12,13,14,15
2,8,9,10,11
1,4,5,6,7
4,16,17,18,19


In [102]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3


In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

### Computing Indicator/Dummy Variables

In [ ]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'])

In [ ]:
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

In [104]:
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

<ipython-input-104-48314dfb286d>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets/movielens/movies.dat', sep='::',


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [225]:
all_genres = []
for x in movies.genres:
    all_genres.extend(x.split('|'))
print('length all_genres', len(all_genres), len(movies))
genres = pd.unique(all_genres)

length all_genres 6408 3883


In [108]:
type(all_genres)

list

In [109]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [110]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

In [113]:
gen = movies.genres[1]
gen.split('|')
dummies.columns.get_indexer(gen.split('|'))

array([3, 1, 4])

In [114]:
# one hot encoding for multiple label categories.
# for each row in the movies dataset, get the indicies in the dummies columns corresponding to the genres.
# Then set those columns to 1
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

In [117]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

movie_id                                      1
title                          Toy Story (1995)
genres              Animation|Children's|Comedy
Genre_Animation                               1
Genre_Children's                              1
                               ...             
Genre_War                                     0
Genre_Musical                                 0
Genre_Mystery                                 0
Genre_Film-Noir                               0
Genre_Western                                 0
Name: 0, Length: 21, dtype: object

In [144]:
# How to do the same operation with numpy?

# stumped

def test_func(arr):
    return arr[0].split()

movies_np = movies.to_numpy()
b = movies_np[:,2]
b.shape
type(b[0])
c = np.apply_along_axis(test_func,0,b)
c

array(["Animation|Children's|Comedy"], dtype='<U27')

In [146]:
np.random.seed(12345)
values = np.random.rand(10)
values
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation

### String Object Methods

In [145]:
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [147]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [150]:
first, second, third = pieces
first + '::' + second + '::' + third

'a::b::guido'

In [151]:
'::'.join(pieces)

'a::b::guido'

In [156]:
'guido' in val
val.index(',')
val.find(':')
val.index('g')

6

In [157]:
val.index(':')

ValueError: substring not found

In [158]:
val.count(',')

2

In [159]:
val.replace(',', '::')
val.replace(',', '')

'ab  guido'

### Regular Expressions

In [171]:
import re
text = "foo    bar\t baz  \tqux"
re.split('\s+', text)
re.findall('\s{1,4}', text)

['    ', '\t ', '  \t']

In [172]:
regex = re.compile('\s+')
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [173]:
regex.findall(text)

['    ', '\t ', '  \t']

In [201]:
text = """Dave dave@google.com
Steve steve@gmail.com  bill@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [202]:
regex.findall(text)

['dave@google.com',
 'steve@gmail.com',
 'bill@gmail.com',
 'rob@gmail.com',
 'ryan@yahoo.com']

In [203]:
m = regex.search(text)
m
text[m.start():m.end()]


'dave@google.com'

In [204]:
print(regex.match(text))

None


In [205]:
print(regex.sub('REDACTED', text))

Dave REDACTED
Steve REDACTED  REDACTED
Rob REDACTED
Ryan REDACTED



In [206]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [207]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [208]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('bill', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [200]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### Vectorized String Functions in pandas

In [209]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [210]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [212]:
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [215]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [216]:
matches.str.get(1)
matches.str[0]

AttributeError: Can only use .str accessor with string values!

In [217]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [218]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS

In [221]:
data.str.split('e')

Dave     [dav, @googl, .com]
Steve    [st, v, @gmail.com]
Rob          [rob@gmail.com]
Wes                      NaN
dtype: object

In [222]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

## Conclusion